In [11]:
import mysql.connector
import pandas as pd


mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="leave_data"
)

mycursor = mydb.cursor()

query = "SELECT * FROM leaves" 
  
mycursor.execute(query)

results = mycursor.fetchall()

df = pd.DataFrame(results, columns=[i[0] for i in mycursor.description])

df.head(10)

,ID,empname,leavetype,applied,from,to,desg,status,status1,status2,...,aprtime,aprremark,hrtime,leavetype2,mgrremark,mgrtime,mgrname,work_location,modified,leavebal
0,2,SURAMPUDI SRINU,HALF DAY,2024-01-08 10:43:58,2024-01-08 14:00:00,2024-01-08 18:00:00,STORE KEEPER,1,1,1,...,2024-01-08 06:36:49,,NaT,AN,None,NaT,None,Visakhapatnam,None,None
1,3,ABHILASH KUMAR SAHU,CASUAL LEAVE,2024-01-08 11:06:15,2024-01-10 00:00:00,2024-01-10 00:00:00,DESIGN ENGINEER,1,1,0,...,2024-01-08 06:36:54,,NaT,0,None,NaT,None,Visakhapatnam,None,None
2,4,THUMPALA ANAND,HALF DAY,2024-01-08 12:02:59,2024-01-08 14:00:00,2024-01-08 18:00:00,MAINTENANCE SUPERVISOR,1,1,1,...,2024-01-08 06:36:51,,NaT,AN,None,NaT,None,Visakhapatnam,None,None
3,5,BONDI MANOHAR,CASUAL LEAVE,2024-01-08 12:11:49,2024-01-09 00:00:00,2024-01-09 00:00:00,QC Engineer,1,1,0,...,2024-01-10 13:40:50,dd,NaT,0,None,NaT,None,Visakhapatnam,None,None
4,6,SURAMPUDI SRINU,CASUAL LEAVE,2024-01-08 14:31:07,2024-01-02 00:00:00,2024-01-02 00:00:00,STORE KEEPER,1,1,1,...,NaT,,NaT,,None,NaT,None,Visakhapatnam,None,None
5,8,THUMPALA ANAND,SICK LEAVE,2024-01-09 09:31:27,2024-01-09 00:00:00,2024-01-09 00:00:00,MAINTENANCE SUPERVISOR,1,1,1,...,2024-01-10 07:23:41,,2024-01-09 04:19:56,,None,NaT,None,Visakhapatnam,None,None
6,10,PEDDINTI VENKATESH,CASUAL LEAVE,2024-01-09 10:52:55,2024-01-04 00:00:00,2024-01-04 00:00:00,ASSEMBLER,1,1,1,...,NaT,,NaT,,None,NaT,None,Visakhapatnam,None,None
7,11,KUNDRAPU MURALI,SICK LEAVE,2024-01-09 10:54:27,2024-01-05 00:00:00,2024-01-06 00:00:00,ASST QC,1,1,1,...,NaT,,NaT,,None,NaT,None,Visakhapatnam,None,None
8,12,DADI PRASAD,CASUAL LEAVE,2024-01-09 10:55:39,2024-01-06 00:00:00,2024-01-06 00:00:00,ASSEMBLER,1,1,1,...,NaT,,NaT,,None,NaT,None,Visakhapatnam,None,None
9,13,SURAMPUDI LAXMAN,CASUAL LEAVE,2024-01-09 10:57:40,2024-01-02 00:00:00,2024-01-02 00:00:00,ASSEMBLER,1,1,1,...,NaT,,NaT,,None,NaT,None,Visakhapatnam,None,None


In [12]:
pip install ipywidgets mysql-connector-python pandas

Note: you may need to restart the kernel to use updated packages.


In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd

# Database connection setup
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",         # Replace with your database host
        user="root",              # Replace with your database user
        password="",      # Replace with your database password
        database="leave_data"       # Replace with your database name
    )

# Fetch leave data based on email
def fetch_leave_data(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        query = """
        SELECT l.id,lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark,l.from,l.to,l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        # ORDER BY id DESC 
        # LIMIT 1
        """
        cursor.execute(query, (email,))
        result = cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Form submission callback
def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        data = fetch_leave_data(email)
        
        if data is not None and not data.empty:
            print("Leave Data:")
            display(data)
        else:
            print("No leave data found for the provided email.")

# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave', 'On Duty'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")

# Create a date picker widget for "From" input

from_date_input = widgets.DatePicker(
    description="From Date",
    disabled=False
)

# Create a date picker widget for "To" input
to_date_input = widgets.DatePicker(
    description="To Date",
    disabled=False
)


reason_input = widgets.Text(description="Reason:")

submit_button = widgets.Button(description="Fetch Data")
output = widgets.Output()

# Attach event listener to the button
submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([leave_type_input,email_input,from_date_input,to_date_input,reason_input, submit_button, output]))


In [14]:
import pandas as pd
df = pd.read_csv('type_of_leave.csv')
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

FileNotFoundError: [Errno 2] No such file or directory: 'type_of_leave.csv'

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Assuming the dataset is a CSV file
# Load the dataset
df = pd.read_csv(r"c:\HarshaPriya\ML\type_of_leave.csv")  # Adjust path to your dataset

# Preprocessing function
def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    words = text.split()  # Tokenization
    
    # Lemmatization and removing stopwords
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return ' '.join(words)

# Apply preprocessing to the 'text' column in the dataset
df['processed_text'] = df['text'].apply(preprocess)

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)

# Transform the 'processed_text' into TF-IDF features
X = vectorizer.fit_transform(df['processed_text'])

# Target labels (y) would be from the 'label' column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression and SVM models
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
svm_model = SVC(kernel='linear', random_state=42)

# Train the models
log_reg_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Make predictions on the test set
log_reg_pred = log_reg_model.predict(X_test)
svm_pred = svm_model.predict(X_test)

# Print accuracy and classification report for both models
print("Logistic Regression Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, log_reg_pred):.4f}")
print(classification_report(y_test, log_reg_pred))

print("SVM Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, svm_pred):.4f}")
print(classification_report(y_test, svm_pred))

# Hyperparameter tuning for SVM using GridSearchCV (Optional)
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_svm_model = grid_search.best_estimator_

# Evaluate the best SVM model
best_svm_pred = best_svm_model.predict(X_test)
print(f"Best SVM Model Accuracy: {accuracy_score(y_test, best_svm_pred):.4f}")
print(classification_report(y_test, best_svm_pred))

# Function to predict whether a given text is a sick leave or not
def predict_sick_leave(text, model, vectorizer):
    # Preprocess the input text
    processed_text = preprocess(text)
    
    # Convert the text to the feature vector
    text_vector = vectorizer.transform([processed_text])
    
    # Predict using the trained model
    prediction = model.predict(text_vector)
    
    # Return result
    return "Sick Leave" if prediction[0] == 1 else "Not Sick Leave"

# Example usage: Get prediction for a new text
new_text = "I met with an accident so i need leave"  # Modify as needed
print(predict_sick_leave(new_text, log_reg_model, vectorizer))  # Using Logistic Regression model


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Logistic Regression Evaluation:
Accuracy: 0.9969
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       150
           1       0.99      1.00      1.00       170

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320

SVM Evaluation:
Accuracy: 0.9969
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       150
           1       0.99      1.00      1.00       170

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320

Best SVM Model Accuracy: 0.9969
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       150
           1       0.99      1.00      1.00       170

    accuracy                           1.00       320
   macro avg  

AttributeError: 'Text' object has no attribute 'lower'

Feature-1


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



In [ ]:
# Database connection setup
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",         # Replace with your database host
        user="root",              # Replace with your database user
        password="",              # Replace with your database password
        database="leave_data"     # Replace with your database name
    )



In [ ]:
# Fetch leave data based on email
def fetch_leave_data(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        query = """
        SELECT l.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark, l.from, l.to, l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        """
        cursor.execute(query, (email,))
        result = cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    





In [ ]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the dataset for leave classification
df = pd.read_csv(r"c:\HarshaPriya\ML\type_of_leave.csv")  # Adjust path to your dataset

# Preprocessing function
def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    words = text.split()  # Tokenization
    unwanted_keywords = ["shopping", "movie", "beach","feeling"]
    words = [word for word in words if word not in unwanted_keywords]
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column in the dataset
df['processed_text'] = df['text'].apply(preprocess)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)


In [ ]:

# Transform the 'processed_text' into TF-IDF features
X = vectorizer.fit_transform(df['processed_text'])

# Target labels (y) would be from the 'label' column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression and SVM models
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
svm_model = SVC(kernel='linear', random_state=42)

# Train the models
log_reg_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)



SVC(kernel='linear', random_state=42)

In [ ]:
# Function to predict whether a given text is a sick leave or not
def predict_sick_leave(text, model, vectorizer):
    # Preprocess the input text
    processed_text = preprocess(text)
    # Convert the text to the feature vector
    text_vector = vectorizer.transform([processed_text])
    # Predict using the trained model
    prediction = model.predict(text_vector)
    # Return result
    if prediction[0] == 1:
        return "Sick Leave"
    else:
        return "Not Sick Leave"

# Form submission callback
def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        
        # Get the leave type selected by the user
        selected_leave_type = leave_type_input.value
        
        # Predict the leave type based on the reason entered
        leave_reason = reason_input.value
        leave_status = predict_sick_leave(leave_reason, log_reg_model, vectorizer)
        
        # Check if the leave type matches the prediction
        if selected_leave_type =="Sick Leave" and leave_status!="Sick Leave":
            print("Leave type and reason do not match.")
            return  # Do not fetch leave data if the leave type and reason don't match
        elif selected_leave_type ==" Casual Leave" and leave_status !="Sick Leave":
            print("leave granted")
        elif selected_leave_type == 'Sick Leave' and leave_status == "Sick Leave":
            print("Leave Granted")
        
        # If leave type matches the predicted reason, fetch the leave data
        data = fetch_leave_data(email)
        
        # Check if email corresponds to a valid user
        if data is not None and not data.empty:
            print("Leave Data:")
            display(data)
            # print(f"Predicted Leave Status: {leave_status}")
            
        else:
            print("User not found.")

# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave', 'On Duty'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")
from_date_input = widgets.DatePicker(description="From Date", disabled=False)
to_date_input = widgets.DatePicker(description="To Date", disabled=False)
reason_input = widgets.Text(description="Reason:")
submit_button = widgets.Button(description="Fetch Data")
output = widgets.Output()

# Attach event listener to the button
submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([leave_type_input, email_input, from_date_input, to_date_input, reason_input, submit_button, output]))


Feature 2

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd

In [ ]:
# Database connection setup
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",         # Replace with your database host
        user="root",              # Replace with your database user
        password="",              # Replace with your database password
        database="leave_data"     # Replace with your database name
    )


In [12]:
# Fetch leave data based on email
def fetch_leave_data(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        query = """
        SELECT l.id,lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark,l.from,l.to,l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        # ORDER BY id DESC 
        # LIMIT 1
        """
        cursor.execute(query, (email,))
        result = cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    
# Check leave eligibility    
def can_apply_leave(data):
    if data is not None and not data.empty:
        # Convert columns to numeric, handling non-numeric values
        data["cl"] = pd.to_numeric(data["cl"], errors='coerce').fillna(0)
        data["sl"] = pd.to_numeric(data["sl"], errors='coerce').fillna(0)
        data["co"] = pd.to_numeric(data["co"], errors='coerce').fillna(0)
        
        # Calculate total leave balance (lb = cl + sl + co)
        data["lb"] = data["cl"] + data["sl"] + data["co"]
        
        # Check if leave can be applied (i.e., if total balance > 0)
        data["can_apply_leave"] = data["lb"] > 0
        return data
    else:
        return "not having enough leave balance"
    
# Form submission callback

def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        data = fetch_leave_data(email)
        
        if data is not None and not data.empty:
            # Check leave eligibility (whether user has enough balance)
            data = can_apply_leave(data)
            
            # If the employee can apply for leave
            if data["can_apply_leave"].iloc[0]:  # Check the first entry's eligibility
                print("Leave Data:")
                display(data)
            else:
                print("You do not have enough leave balance to apply for leave.")
        else:
            print("No leave data found for the provided email.")


# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave', 'On Duty'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")

# Create a date picker widget for "From" input

from_date_input = widgets.DatePicker(
    description="From Date",
    disabled=False
)

# Create a date picker widget for "To" input
to_date_input = widgets.DatePicker(
    description="To Date",
    disabled=False
)


reason_input = widgets.Text(description="Reason:")

submit_button = widgets.Button(description="Fetch Data")
output = widgets.Output()

# Attach event listener to the button
submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([leave_type_input,email_input,from_date_input,to_date_input,reason_input, submit_button, output]))


Combining Feature1 and Feauture2

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



In [2]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="leave_data"
    )

In [4]:
def fetch_leave_data(email):
    try:
        conn=connect_to_db()
        cursor=conn.cursor(dictionary=True)
        query="""
        SELECT l.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark, l.from, l.to, l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        """
        cursor.execute(query,(email,))
        result=cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error:{err}")
        return None

In [5]:
#leave eligibility

def can_apply_leave(data):
    if data is not None and not data.empty:
        # Convert columns to numeric, handling non-numeric values
        data["cl"] = pd.to_numeric(data["cl"], errors='coerce').fillna(0)
        data["sl"] = pd.to_numeric(data["sl"], errors='coerce').fillna(0)
        data["co"] = pd.to_numeric(data["co"], errors='coerce').fillna(0)
        
        # Calculate total leave balance (lb = cl + sl + co)
        data["lb"] = data["cl"] + data["sl"] + data["co"]
        
        # Check if leave can be applied (i.e., if total balance > 0)
        data["can_apply_leave"] = data["lb"] > 0
        return data
    else:
        return "not having enough leave balance"


#text preprocessing

In [6]:
 

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the dataset for leave classification
df = pd.read_csv(r"c:\HarshaPriya\ML\type_of_leave.csv")  # Adjust path to your dataset

# Preprocessing function
def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    words = text.split()  # Tokenization
    unwanted_keywords = ["feeling"]
    words = [word for word in words if word not in unwanted_keywords]
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column in the dataset
df['processed_text'] = df['text'].apply(preprocess)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)


In [8]:

# Transform the 'processed_text' into TF-IDF features
X = vectorizer.fit_transform(df['processed_text'])

# Target labels (y) would be from the 'label' column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression and SVM models
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
svm_model = SVC(kernel='linear', random_state=42)

# Train the models
log_reg_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)



SVC(kernel='linear', random_state=42)

In [9]:
# Function to predict whether a given text is a sick leave or not
def predict_sick_leave(text, model, vectorizer):
    # Preprocess the input text
    processed_text = preprocess(text)
    # Convert the text to the feature vector
    text_vector = vectorizer.transform([processed_text])
    # Predict using the trained model
    prediction = model.predict(text_vector)
    # Return result
    if prediction[0] == 1:
        return "Sick Leave"
    else:
        return "Not Sick Leave"
# Form submission callback

def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        data = fetch_leave_data(email)
        selected_leave_type = leave_type_input.value
        leave_reason = reason_input.value
        leave_status = predict_sick_leave(leave_reason, log_reg_model, vectorizer)

        if data is not None and not data.empty:
            # Check leave eligibility (whether user has enough balance)
            data = can_apply_leave(data)
            
            
            if data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                print("Leave Granted")  
                
            elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                print("Leave Granted")  
                
            elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                print("Leave Granted")  

            
            elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                print("You do not have enough leave balance to apply for leave.")
                
            
            elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                
            elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                print("Leave type isn't matching with reason.")
                
            elif selected_leave_type == "Sick Leave" and leave_status != "Sick Leave":
                print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                return 
                
            else:
                print("You do not have enough leave balance to apply for leave.")
        else:
            print("No leave data found for the provided email.")



# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave', 'On Duty'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")

# Create a date picker widget for "From" input

from_date_input = widgets.DatePicker(
    description="From Date",
    disabled=False
)

# Create a date picker widget for "To" input
to_date_input = widgets.DatePicker(
    description="To Date",
    disabled=False
)


reason_input = widgets.Text(description="Reason:")

submit_button = widgets.Button(description="Fetch Data")
output = widgets.Output()

# Attach event listener to the button
submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([leave_type_input,email_input,from_date_input,to_date_input,reason_input, submit_button, output]))

Feature 3

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd
from datetime import date

# Database connection setup
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",         # Replace with your database host
        user="root",              # Replace with your database user
        password="",              # Replace with your database password
        database="leave_data"     # Replace with your database name
    )

# Fetch leave data based on email
def fetch_leave_data(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        query = """
        SELECT l.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, 
               l.hrremark, l.mgrremark, l.aprremark, l.from, l.to, l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        """
        cursor.execute(query, (email,))
        result = cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Preprocessing function (to be defined based on your use case)
def preprocess(text):
    # Example of simple preprocessing (this can be expanded)
    return text.lower()

# Function to predict whether a given text is a sick leave or not
def predict_sick_leave(text, model, vectorizer):
    # Preprocess the input text
    processed_text = preprocess(text)
    # Convert the text to the feature vector
    text_vector = vectorizer.transform([processed_text])
    # Predict using the trained model
    prediction = model.predict(text_vector)
    # Return result
    if prediction[0] == 1:
        return "Sick Leave"
    else:
        return "Not Sick Leave"

# Can apply leave check (check for leave eligibility)
def can_apply_leave(data):
    # This function should be implemented based on your actual logic
    # For demonstration, we assume leave can always be applied
    data['can_apply_leave'] = True
    return data

# Form submission callback
def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        from_date = from_date_input.value
        to_date = to_date_input.value
        selected_leave_type = leave_type_input.value
        leave_reason = reason_input.value
        leave_status = predict_sick_leave(leave_reason, log_reg_model, vectorizer)  # Assume model and vectorizer are available

        if not from_date or not to_date:
            print("Please select both 'From Date' and 'To Date'.")
            return

        if from_date < today or to_date < today:
            print("Dates must be today or in the future. Please select valid dates.")
            return

        if from_date > to_date:
            print("'From Date' cannot be later than 'To Date'. Please correct the dates.")
            return

        delta = (to_date - from_date).days + 1  

        if delta > 6:
            print("You cannot apply for leave more than 6 days.")
        else:
            data = fetch_leave_data(email)
            if data is not None and not data.empty:
                data = can_apply_leave(data)
                
                # Check leave eligibility (whether user has enough balance)
                if data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    print("Leave Granted")  

                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    print("Leave Granted")  

                elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    print("Leave Granted")  

                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    print("You do not have enough leave balance to apply for leave.")
                
                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                    print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                
                elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                    print("Leave type isn't matching with reason.")
                
                elif selected_leave_type == "Sick Leave" and leave_status != "Sick Leave":
                    print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                    return 
                
                else:
                    print("You do not have enough leave balance to apply for leave.")
            else:
                print("No leave data found for the provided email.")

# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave', 'On Duty'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")

# Create date picker widgets for "From" and "To" inputs with min date set to today
today = date.today()
from_date_input = widgets.DatePicker(
    description="From Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)
to_date_input = widgets.DatePicker(
    description="To Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)

reason_input = widgets.Text(description="Reason:")

submit_button = widgets.Button(description="Fetch Data")
output = widgets.Output()

# Attach event listener to the button
submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([ 
    leave_type_input, 
    email_input, 
    from_date_input, 
    to_date_input, 
    reason_input, 
    submit_button, 
    output 
]))


Combining Feature 1, 2 and 3

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



In [2]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="leave_data"
    )

In [3]:
def fetch_leave_data(email):
    try:
        conn=connect_to_db()
        cursor=conn.cursor(dictionary=True)
        query="""
        SELECT l.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark, l.from, l.to, l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        """
        cursor.execute(query,(email,))
        result=cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error:{err}")
        return None

In [4]:
#leave eligibility

def can_apply_leave(data):
    if data is not None and not data.empty:
        # Convert columns to numeric, handling non-numeric values
        data["cl"] = pd.to_numeric(data["cl"], errors='coerce').fillna(0)
        data["sl"] = pd.to_numeric(data["sl"], errors='coerce').fillna(0)
        data["co"] = pd.to_numeric(data["co"], errors='coerce').fillna(0)
        
        # Calculate total leave balance (lb = cl + sl + co)
        data["lb"] = data["cl"] + data["sl"] + data["co"]
        
        # Check if leave can be applied (i.e., if total balance > 0)
        data["can_apply_leave"] = data["lb"] > 0
        return data
    else:
        return "not having enough leave balance"


In [5]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the dataset for leave classification
df = pd.read_csv(r"c:\HarshaPriya\ML\type_of_leave.csv")  # Adjust path to your dataset

# Preprocessing function
def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    words = text.split()  # Tokenization
    unwanted_keywords = ["feeling"]
    words = [word for word in words if word not in unwanted_keywords]
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column in the dataset
df['processed_text'] = df['text'].apply(preprocess)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)


In [7]:

# Transform the 'processed_text' into TF-IDF features
X = vectorizer.fit_transform(df['processed_text'])

# Target labels (y) would be from the 'label' column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression and SVM models
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
svm_model = SVC(kernel='linear', random_state=42)

# Train the models
log_reg_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)



SVC(kernel='linear', random_state=42)

In [8]:
from datetime import date
# Function to predict whether a given text is a sick leave or not
def predict_sick_leave(text, model, vectorizer):
    # Preprocess the input text
    processed_text = preprocess(text)
    # Convert the text to the feature vector
    text_vector = vectorizer.transform([processed_text])
    # Predict using the trained model
    prediction = model.predict(text_vector)
    # Return result
    if prediction[0] == 1:
        return "Sick Leave"
    else:
        return "Not Sick Leave"

# Form submission callback
def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        data = fetch_leave_data(email)
        from_date = from_date_input.value
        to_date = to_date_input.value
        selected_leave_type = leave_type_input.value
        leave_reason = reason_input.value
        leave_status = predict_sick_leave(leave_reason, log_reg_model, vectorizer)  # Assume model and vectorizer are available

        if not from_date or not to_date:
            print("Please select both 'From Date' and 'To Date'.")
            return

        elif from_date < today or to_date < today:
            print("Dates must be today or in the future. Please select valid dates.")
            return

        elif from_date > to_date:
            print("'From Date' cannot be later than 'To Date'. Please correct the dates.")
            return

        delta = (to_date - from_date).days + 1  

        if delta > 6:
            print("You cannot apply for leave more than 6 days.")
        else:
            if data is not None and not data.empty:
            # Check leave eligibility (whether user has enough balance)
                data = can_apply_leave(data)
                
                
                if data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    print("Leave Granted")  
                    
                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    print("Leave Granted")  
                    
                elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    print("Leave Granted")  

                
                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    print("You do not have enough leave balance to apply for leave.")
                    
                
                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                    print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                    
                elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                    print("Leave type isn't matching with reason.")
                    
                elif selected_leave_type == "Sick Leave" and leave_status != "Sick Leave":
                    print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                    return 
                    
                else:
                    print("You do not have enough leave balance to apply for leave.")
            else:
                print("No leave data found for the provided email.")

# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave', 'On Duty'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")

# Create date picker widgets for "From" and "To" inputs with min date set to today
today = date.today()
from_date_input = widgets.DatePicker(
    description="From Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)
to_date_input = widgets.DatePicker(
    description="To Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)

reason_input = widgets.Text(description="Reason:")

submit_button = widgets.Button(description="Fetch Data")
output = widgets.Output()

# Attach event listener to the button
submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([ 
    leave_type_input, 
    email_input, 
    from_date_input, 
    to_date_input, 
    reason_input, 
    submit_button, 
    output 
]))